In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/Drive')
df = pd.read_csv('/Drive/MyDrive/Projet MLE/DisneylandReviews.csv', encoding='cp1252')

Drive already mounted at /Drive; to attempt to forcibly remount, call drive.mount("/Drive", force_remount=True).


SAVE STOPWORDS and TOKENIZER

In [ ]:
import pickle
import nltk
!pip install nltk==3.5
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import NLTKWordTokenizer

stop_words = set(stopwords.words('english'))
stop_words.update(["'ve", "", "'ll", "'s", ".", ",", "?", "!", "(", ")", "..", "'m", "n", "u"])

path='/Drive/My Drive/Projet MLE'
f = open(f'{path}/stopwords.pkl',"wb")
pickle.dump(stop_words,f)
f.close()

tokenizer = NLTKWordTokenizer()
f = open(f'{path}/NLTKWordTokenizer.pkl',"wb")
pickle.dump(tokenizer,f)
f.close()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: ignored

LOAD STOPWORDS and TOKENIZER

In [ ]:
!pip install nltk==3.5
import nltk
from nltk.corpus import stopwords
import pickle
from nltk.tokenize import NLTKWordTokenizer
path='/Drive/My Drive/Projet MLE'

with open(f'{path}/stopwords.pkl', 'rb') as handle:
    pkl_stopwords = pickle.load(handle)
with open(f'{path}/NLTKWordTokenizer.pkl', 'rb') as handle:
    pkl_tokenizer = pickle.load(handle)

# Projet MLE

> Le but de ce projet est de mettre en production un modèle d'analyse de sentiment construit sur le jeu de données de commentaires sur [Disneyland](https://www.kaggle.com/arushchillar/disneyland-reviews).

KAGGLE

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv', encoding='cp1252')

FileNotFoundError: ignored

In [ ]:
df.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42656 entries, 0 to 42655
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          42656 non-null  int64 
 1   Rating             42656 non-null  int64 
 2   Year_Month         42656 non-null  object
 3   Reviewer_Location  42656 non-null  object
 4   Review_Text        42656 non-null  object
 5   Branch             42656 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [ ]:
df['Rating'].unique()

array([4, 3, 5, 2, 1])

# Préparation des données

In [ ]:
df = df.drop(['Review_ID', 'Year_Month', 'Reviewer_Location'], axis=1)
df_original = df.copy(deep=True)

In [ ]:
df.head(5)

,Rating,Review_Text,Branch
0,4,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,4,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,4,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,4,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,4,"the location is not in the city, took around 1...",Disneyland_HongKong


In [ ]:
"""
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import NLTKWordTokenizer
"""
def preprocess_text(text):
    #stop_words = set(stopwords.words('english'))
    #stop_words.update(["'ve", "", "'ll", "'s", ".", ",", "?", "!", "(", ")", "..", "'m", "n", "u"])
    
    #tokenizer = NLTKWordTokenizer()
    
    text = text.lower()
    
    tokens = pkl_tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in pkl_stopwords]
    
    return ' '.join(tokens)


df_original = df.copy(deep=True)
df['Review_Text'] = df['Review_Text'].apply(preprocess_text)
df.head()

,Rating,Review_Text,Branch
0,4,ever disneyland anywhere find disneyland hong ...,Disneyland_HongKong
1,4,since last time visit hk disneyland yet time s...,Disneyland_HongKong
2,4,thanks god hot humid visiting park otherwise w...,Disneyland_HongKong
3,4,hk disneyland great compact park. unfortunatel...,Disneyland_HongKong
4,4,location city took around 1 hour kowlon kids l...,Disneyland_HongKong


In [ ]:
df_original.head()

,Rating,Review_Text,Branch
0,4,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,4,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,4,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,4,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,4,"the location is not in the city, took around 1...",Disneyland_HongKong


# Premier Modele

> Le premier modèle consiste à considérer toutes les branches ensemble.

In [ ]:
df1 = df.drop(['Branch'], axis=1)
df1.head()


,Rating,Review_Text
0,4,ever disneyland anywhere find disneyland hong ...
1,4,since last time visit hk disneyland yet time s...
2,4,thanks god hot humid visiting park otherwise w...
3,4,hk disneyland great compact park. unfortunatel...
4,4,location city took around 1 hour kowlon kids l...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split

features = df['Review_Text']
target = df['Rating']

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=123)

count_vectorizer_unique = CountVectorizer(max_features=2000)

X_train_cv = count_vectorizer_unique.fit_transform(X_train)
"""f = open(f'{path}/count_vectorizer1.pkl',"wb")
pickle.dump(count_vectorizer_unique,f)
f.close()"""

'f = open(f\'{path}/count_vectorizer1.pkl\',"wb")\npickle.dump(count_vectorizer_unique,f)\nf.close()'

In [ ]:
with open(f'{path}/count_vectorizer1.pkl', 'rb') as handle:
    pkl_count_vectorizer1 = pickle.load(handle)
X_test_cv = pkl_count_vectorizer1.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# model_unique = RandomForestClassifier(max_depth=3, n_estimators=100)
model_unique = LogisticRegression()
# model_unique = DecisionTreeClassifier(max_depth=8)

model_unique.fit(X_train_cv, y_train)
"""f = open(f'{path}/model1.pkl',"wb")
pickle.dump(model_unique,f)
f.close()"""


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


'f = open(f\'{path}/model1.pkl\',"wb")\npickle.dump(model_unique,f)\nf.close()'

In [ ]:
with open(f'{path}/model1.pkl', 'rb') as handle:
    pkl_model1 = pickle.load(handle)
pkl_model1.score(X_test_cv, y_test)
ypred=pkl_model1.predict(X_test_cv)

In [ ]:
print("score:",pkl_model1.score(X_test_cv, y_test))

score: 0.6110277569392348


In [ ]:
X_test

9591     read much bad publicity expecting bad disneyla...
40108    spent two days got round rides.worth checking ...
2197     kids go crazy plenty restaurants & snack stall...
23202    busy busy day long queues rides. fast passes a...
37038    great time husband 3yr old daughter myself. bu...
                               ...                        
8982     second visit hong kong disneyland still holds ...
1759     place beautiful rides good long queues.it may ...
202      visit magical last. son always enjoys meeting ...
22898    disney ... say amazing long time enthusiast mu...
37873    husband finally chance attend disneyland paris...
Name: Review_Text, Length: 10664, dtype: object

In [ ]:
X_test[np.logical_and(ypred==3, y_test==3) ]

9831     walt disney currently cryogenic chamber fire s...
12357    think way exitced come disneyland. held expect...
10227    first time la took kids disney. great time dis...
13292    waited years go however disappointed need upda...
42016    name eurodisney describes geographic location ...
                               ...                        
120      florida anaheim euro disney hong kong disney d...
33417    bearing mind expensive leisure activity sooooo...
9713     paid quite lot admission parking expected take...
22660    disney parks orlando paris found one least att...
1759     place beautiful rides good long queues.it may ...
Name: Review_Text, Length: 433, dtype: object

In [ ]:
df_original.iloc[9831]

Rating                                                         3
Review_Text    if walt disney is currently in a cryogenic cha...
Branch                                     Disneyland_California
Name: 9831, dtype: object

In [ ]:
df_original.iloc[9831]['Review_Text']

'if walt disney is currently in a cryogenic chamber above the fire station in disneyland   s main street, he must be wanting to pull the plug. having been a fan and visitor of orlando   s walt disney world, disneyland was a disappointment. most of the staff seems to hate working there, it was hard to receive a smile. i don   t blame them, they have recently had disputes with they   re salary. the park was mostly dirty, and the bathrooms in 7 eleven were cleaner that most of the restrooms across the parks. about 40% of the rides had issues midway through. and they were unannounced closures throughout the day.one good thing was the 3 day passes that included maxpass (for fast passes) and photo pass, which were a great value. i went in trying to feel like a kid, but they brought the adult side of me. '

In [ ]:
text="Visited 21 5 2014. This park is a joke, three main rides were closed in one park and one in the other and two broken which means the huge crowds made for 2 hour wait times. At $150 for a one day hopper this turned out to be an expensive crowded walk. Google Disneyland to see what a shameful joke this place is, oh...$17 to park a car. I have been to Disney Florida for 29 years so I am not naive about wait times, this park is a disgrace."
text=[preprocess_text(text)]
text_cv = pkl_count_vectorizer1.transform(text)
print(pkl_model1.predict(text_cv))
text="if walt disney is currently in a cryogenic chamber above the fire station in disneyland   s main street, he must be wanting to pull the plug. having been a fan and visitor of orlando   s walt disney world, disneyland was a disappointment. most of the staff seems to hate working there, it was hard to receive a smile. i don   t blame them, they have recently had disputes with they   re salary. the park was mostly dirty, and the bathrooms in 7 eleven were cleaner that most of the restrooms across the parks. about 40% of the rides had issues midway through. and they were unannounced closures throughout the day.one good thing was the 3 day passes that included maxpass (for fast passes) and photo pass, which were a great value. i went in trying to feel like a kid, but they brought the adult side of me."
text=[preprocess_text(text)]
text_cv = pkl_count_vectorizer1.transform(text)
print(pkl_model1.predict(text_cv))

[1]
[3]


## Deuxieme modele

> Dans ce modele les branches sont séparées en 3.

In [ ]:
df['Branch'].unique()

array(['Disneyland_HongKong', 'Disneyland_California', 'Disneyland_Paris'],
      dtype=object)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

count_vectorizers = {}
models = {}
#df = pd.read_csv('/Drive/MyDrive/Projet MLE/DisneylandReviews.csv', encoding='cp1252')
#df = df.drop(['Review_ID', 'Year_Month', 'Reviewer_Location'], axis=1)

index=2
for branch in df['Branch'].unique():
    print(branch)
    count_vectorizer = CountVectorizer(max_features=2000)
#     model = LogisticRegression()
    model = RandomForestClassifier(n_estimators=20, max_depth=5)
    
    df_temp = df[df['Branch'] == branch]
    X_train, X_test, y_train, y_test = train_test_split(df_temp['Review_Text'], df_temp['Rating'], random_state=123)

    X_train_cv = count_vectorizer.fit_transform(X_train)

    """
    #save count_vectorizer
    f = open(f'{path}/count_vectorizer{index}.pkl',"wb")
    pickle.dump(count_vectorizer,f)
    f.close()"""
    #load count_vectorizer
    with open(f'{path}/count_vectorizer{index}.pkl', 'rb') as handle:
      count_vectorizer = pickle.load(handle) 

    X_test_cv = count_vectorizer.transform(X_test)
    
    model.fit(X_train_cv, y_train)
    
    """
    #save model
    f = open(f'{path}/model{index}.pkl',"wb")
    pickle.dump(model,f)
    f.close()"""
    #load model
    with open(f'{path}/model{index}.pkl', 'rb') as handle:
      model = pickle.load(handle) 
    
    print(branch, ':', model.score(X_test_cv, y_test))
    ypred=model.predict(X_test_cv)
    print(ypred[ypred!=5])
    print(X_test[ypred!=5])
    #print(X_test_cv[np.logical_and(ypred==1, y_test==1) ])
    count_vectorizers[branch] = count_vectorizer
    models[branch] = model
    
    text="Visited 21 5 2014. This park is a joke, three main rides were closed in one park and one in the other and two broken which means the huge crowds made for 2 hour wait times. At $150 for a one day hopper this turned out to be an expensive crowded walk. Google Disneyland to see what a shameful joke this place is, oh...$17 to park a car. I have been to Disney Florida for 29 years so I am not naive about wait times, this park is a disgrace."
    text=[preprocess_text(text)]
    text_cv = count_vectorizers[branch].transform(text)
    print("predict1:", models[branch].predict(text_cv))
    text="if walt disney is currently in a cryogenic chamber above the fire station in disneyland   s main street, he must be wanting to pull the plug. having been a fan and visitor of orlando   s walt disney world, disneyland was a disappointment. most of the staff seems to hate working there, it was hard to receive a smile. i don   t blame them, they have recently had disputes with they   re salary. the park was mostly dirty, and the bathrooms in 7 eleven were cleaner that most of the restrooms across the parks. about 40% of the rides had issues midway through. and they were unannounced closures throughout the day.one good thing was the 3 day passes that included maxpass (for fast passes) and photo pass, which were a great value. i went in trying to feel like a kid, but they brought the adult side of me."
    text=[preprocess_text(text)]
    text_cv = count_vectorizers[branch].transform(text)
    print("predict2:", models[branch].predict(text_cv))
    index += 1
    

Disneyland_HongKong
Disneyland_HongKong : 0.4802494802494803
[4 4 4 4 4 4 4 4 4 4 4 4 4]
6765    reviews disneyland talk queue jumpers people p...
6456    booked two day visit via klook.com. visited ja...
9574    reading many differing reviews decided give tr...
8623    disneyland japan compare disney hk much smalle...
1183    18th wedding anniversary decided keep us local...
7006    visited hong kong disneyland multiple times ev...
5332    n't theme park bad hong kong disneyland. usual...
8772    difficult disappointed visiting disneyland hon...
3312    went 2 days. sunday tuesday. experience totall...
370     castle closed street parade pricey food souven...
9456    way back big buddha decided go here. took subw...
9314    8 yo daughter visited disneyland hkg aug. give...
1723    recent trip hong kong another testing user fri...
Name: Review_Text, dtype: object
predict1: [5]
predict2: [5]
Disneyland_California
Disneyland_California : 0.6471558120362737
[3 4]
12236    hahaha title say

In [ ]:
df_original.iloc[8623]['Review_Text']

"I've to been to Disneyland Japan, and compare to this, Disney HK is much smaller, especially the Sleeping Beauty Castle. I visited during weekdays but still the crowd is so many. Mostly the people are from mainland China. Maybe this is summer holiday and the China crowd came. I'm not sure. Very hot and humid during this time of the year. I will go again only if   the place is expanding (there is this new attraction call 'mystic' coming soon next to Toy Story land ; and will only go during cooler season such as autumn or end on winter. Overall, good place to bring your kids."

In [ ]:
df_original.iloc[12236]['Review_Text']

"Hahaha, title says it all. Ridiculous prices, but do you get what you pay for? Eh, maybe yes, maybe no. I went alone so I could do what I wanted to do. This is the first time I've actually paid to go because usually I get free tickets by winning them or knowing people who work there who need to use their extra tickets, and go with family. Prices are ridiculous, but I knew that before going. This is technically a review for both Disney parks. Cons1) Ticket lines are sooooooooooo slow. There were only a couple of people ahead of me and it took forever for my turn. Then I went up, told the lady exactly what I wanted, had my cash out, and it still took her forever. A snail could have done it faster.2) Walking in the gate with the ticket you just bought is just as slow and painful. They take pictures of you now so if you buy a multi day ticket you can't sell it off to someone else (fine, whatever, I get it) but it just makes the lines horrendously long and slow just to get in. The people b

In [ ]:
df_original.iloc[39085]['Review_Text']

"Went to Disneyland Paris, 4 8 September. All I can say is that whatever price you have budgeted for this holiday, treble it at least and that is how much money you are likely to spend.My advice is save up and go to Florida. The food and drink are way over priced, all the fast food is the same. The Queues are ridiculous, 30 minutes every time you want to buy a drink, 2.59 euros for a small cup of tea.The staff have no idea of what customer service is, for example at Casey's Corner, I wanted 5 teas, Queued for ages to be told they had no milk but I could have hot milk in my tea and then was told I would have to pay for the hot milk on top of the 2.59 for the tea, what a joke, I told them to shove it.The Americans fall over themselves backwards to help,the Disney staff need to get some serious tips on customer service. 95% don't crack a smile and couldn't care less if you enjoy your day or not, as long as you are paying the money that's all they seem bothered about.The Queues for rides a

## Objectifs

> L'objectif de ce projet est de déployer les modèles créés plus tôt. Attention, en production, les modèles ne devront pas être relancés.
>
> ### L'API
>
> On va dans un premier construire une API avec Flask ou FastAPI. Cette API devra permettre d'interroger les différents modèles. Les utilisateurs pourront aussi interroger l'API pour accéder aux performances de l'algorithme sur les jeux de tests. Enfin il faut permettre aux utilisateurs d'utiliser une identification basique. (On pourra utiliser le `header` `Authentication` et encoder `username:password` en base 64).
> On pourra utiliser la liste d'utilisateurs/mots de passe suivante:
> - alice: wonderland
> - bob: builder
> - clementine: mandarine
> - ... 
>
> ### Le container
>
> Il s'agira ici de créer un container Docker pour déployer facilement l'API. On portera une attention particulière aux librairies Python à installer ainsi qu'à leurs différentes versions. 
>
> ### Les tests
>
> Une série de tests devra être créée pour tester l'API contenairisée. On pourra pour cela créé un fichier `docker-compose.yml` en s'inspirant de ce qui a été fait dans l'évaluation de Docker. 
>
> ### Kubernetes
>
> On pourra enfin créer un fichier de déploiement ainsi qu'un Service et un Ingress avec Kubernetes pour permettre le déploiement de l'API sur au moins 3 Pods.

## Rendus

> Les attendus sont un fichier pdf contenant des précisions sur les fichiers, sur les différentes étapes ainsi que sur les choix effectués. 
> On devra aussi rendre un repo Github sur lequel seront les fichiers suivants:
> - fichier source de l'API
> - Dockerfile de l'API
> - dans un dossier l'ensemble des fichiers utilisés pour créer les tests
> - les fichiers de déploiements de Kubernetes
> - tout autre fichier ayant été utilisés pour faire ce projet.